In [4]:
import numpy as np
from scipy.interpolate import CubicSpline

# Given data
pnl_grid = {
    506: ([0.465, 0.05, 0.345, 0.629, 0.289, 0.05, 0.243, 0.822, 0.665, 0.856], [-220.983, -217.841, -220.074, -222.224, -219.65, -217.841, np.nan, -223.685, np.nan, -223.942]),
    258: ([0.473, 0.232, 0.649, 0.19, 0.962, 0.93, 0.639, 0.059, 0.831, 0.837], [-653.225, -654.91, -651.994, -655.203, -649.806, -650.03, -652.064, np.nan, -650.722, -650.68]),
    358: ([0.97, 0.836, 0.031, 0.831, 0.634, 0.56, 0.046, 0.094, 0.202, 0.198], [687.434, 686.908, 683.749, 686.888, 686.115, 685.825, 683.808, 683.997, 684.42, 684.405]),
    735: ([0.949, 0.326, 0.205, 0.952, 0.543, 0.032, 0.926, 0.826, 0.875, 0.846], [-426.147, -428.488, -428.943, -426.136, -427.673, -429.593, -426.234, -426.609, np.nan, -426.534]),
    166: ([0.041, 0.48, 0.575, 0.09, 0.412, 0.12, 0.584, 0.306, 0.981, 0.649], [769.374, 762.653, 761.199, 768.624, 763.694, np.nan, 761.061, np.nan, 754.983, 760.066]),
    781: ([0.569, 0.334, 0.102, 0.744, 0.685, 0.546, 0.85, 0.097, 0.791, 0.249], [-449.069, -448.246, np.nan, -449.682, np.nan, -448.989, -450.054, -447.416, -449.847, -447.948]),
    789: ([0.864, 0.536, 0.223, 0.578, 0.646, 0.147, 0.401, 0.535, 0.51, 0.69], [477.718, 472.913, 468.328, 473.528, 474.525, 467.215, 470.936, 472.899, 472.532, 475.169]),
    822: ([0.051, 0.068, 0.386, 0.224, 0.618, 0.969, 0.581, 0.616, 0.405, 0.573], [-999.429, -999.695, -1004.679, -1002.14, -1008.315, -1013.816, -1007.735, -1008.284, -1004.977, -1007.61]),
    728: ([0.605, 0.18, 0.575, 0.316, 0.723, 0.911, 0.98, 0.291, 0.823, 0.63], [336.468, 332.996, 336.223, 334.107, np.nan, 338.968, 339.531, 333.903, 338.249, 336.672]),
    725: ([0.76, 0.703, 0.223, 0.785, 0.211, 0.48, 0.644, 0.551, 0.871, 0.275], [-204.815, -205.658, -212.755, np.nan, -212.933, -208.955, -206.53, -207.905, -203.174, -211.986]),
}

shocks = {
    506: 0.486,
    258: 0.661,
    358: 0.371,
    735: 0.293,
    166: 0.203,
    781: 0.633,
    789: 0.529,
    822: 0.86,
    728: 0.038,
    725: 0.886
}

cutoffs = {
    506: 0.4,
    258: 0.6,
    358: 0.3,
    735: 0.3,
    166: 0.2,
    781: 0.6,
    789: 0.5,
    822: 0.8,
    728: 0.4,
    725: 0.4
}





In [5]:
# Step 1: Remove NaN and infinite values
for issuer_id, (shock_list, pnl_list) in pnl_grid.items():
    valid_indices = np.isfinite(pnl_list)
    pnl_grid[issuer_id] = (np.array(shock_list)[valid_indices], np.array(pnl_list)[valid_indices])

# Step 2: Sort and make lists unique
for issuer_id, (shock_list, pnl_list) in pnl_grid.items():
    sorted_indices = np.argsort(shock_list)
    shock_list_sorted = np.array(shock_list)[sorted_indices]
    pnl_list_sorted = np.array(pnl_list)[sorted_indices]

    unique_indices = np.unique(shock_list_sorted, return_index=True)[1]
    shock_list_unique = shock_list_sorted[unique_indices]
    pnl_list_unique = pnl_list_sorted[unique_indices]

    pnl_grid[issuer_id] = (shock_list_unique, pnl_list_unique)

# Step 3: Interpolate and calculate pnl using CubicSpline
calculated_pnls = {}
for issuer_id, shock in shocks.items():
    shock_list, pnl_list = pnl_grid[issuer_id]
    cs = CubicSpline(shock_list, pnl_list)
    pnl = cs(shock)
    calculated_pnls[issuer_id] = pnl


# Simulation parameters
num_sims = 1000
num_timesteps = 4  # 4 timesteps in a year

# Define cutoffs for each issuer
cutoffs = {
    506: 0.4,
    258: 0.6,
    358: 0.3,
    735: 0.3,
    166: 0.2,
    781: 0.6,
    789: 0.5,
    822: 0.8,
    728: 0.4,
    725: 0.4
}

# Initialize the sum of default PNLs for each issuer
sum_default_pnls = {issuer_id: 0.0 for issuer_id in pnl_grid}

# Simulation loop
for sim in range(num_sims):
    # Initialize default status for each issuer
    default_status = {issuer_id: False for issuer_id in pnl_grid}
    
    # Generate shocks for each issuer at each timestep
    for issuer_id, shock in shocks.items():
        np.random.seed(issuer_id)  # Use issuer id as seed
        issuer_shocks = np.random.uniform(0, 1, num_timesteps)
        
        # Check if issuer defaults at each timestep
        for t in range(num_timesteps):
            if not default_status[issuer_id]:
                if issuer_shocks[t] > cutoffs[issuer_id]:
                    default_status[issuer_id] = True
    
    # Calculate default PNL for each issuer and timestep
    for t in range(num_timesteps):
        for issuer_id, is_default in default_status.items():
            if is_default:
                shock_list, pnl_list = pnl_grid[issuer_id]
                cs = CubicSpline(shock_list, pnl_list)
                default_pnl = cs(shocks[issuer_id])
                sum_default_pnls[issuer_id] += default_pnl
            else:
                sum_default_pnls[issuer_id] += 0.0

# Calculate expected default PNL for each issuer
expected_default_pnls = {issuer_id: total_default_pnl / num_sims for issuer_id, total_default_pnl in sum_default_pnls.items()}

# Print the expected default PNLs
for issuer_id, pnl in expected_default_pnls.items():
    print(f"Issuer ID: {issuer_id}, Expected Default PNL: {pnl:.3f}")


Issuer ID: 506, Expected Default PNL: -884.568
Issuer ID: 258, Expected Default PNL: -2607.640
Issuer ID: 358, Expected Default PNL: 2740.297
Issuer ID: 735, Expected Default PNL: -1714.448
Issuer ID: 166, Expected Default PNL: 3067.577
Issuer ID: 781, Expected Default PNL: -1797.170
Issuer ID: 789, Expected Default PNL: 1891.254
Issuer ID: 822, Expected Default PNL: -4047.600
Issuer ID: 728, Expected Default PNL: 1327.293
Issuer ID: 725, Expected Default PNL: -811.810
